### local dev/testing

#SUDO Code

1. establish db connection
2. check db if previous players & match data exist
 - if so import data to establish season
 - if not: create and save 
3. run engine 

In [51]:
#these would be env vars recieved from the pod.yaml
USER = 'root'          
PASSWORD = 'my-secret-pw'  
DB='SSPDB'
HOST = 'localhost'

In [2]:
import logging
from sqlalchemy import create_engine
import pandas as pd
import synthSportPlayer

In [3]:
#connect to DB
logging.info('Establishing connection to database')
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=HOST, db=DB, user=USER, pw=PASSWORD))

In [4]:
#get table with table names
df = pd.read_sql('SHOW TABLES', con=engine)
display(df)

,Tables_in_SSPDB
0,PLAYER_DATA


In [5]:
df = pd.read_sql('SHOW TABLES', con=engine)
if 'PLAYER_DATA' in df['Tables_in_SSPDB'].values:
    logging.info('Table already exists')
    df2 = pd.read_sql('SELECT * FROM PLAYER_DATA', con=engine)
    pl=[]
    for i in df2.index:
        p=synthSportPlayer.player(int(df2.loc[i]['skill']),int(df2.loc[i]['variance']),df2.loc[i]['name'],int(df2.loc[i]['pointLimit']))
        p.totalPoints=int(df2.loc[i]['totalPoints'])
        if df2.loc[i]['pointRec']!='':
            p.pointRec=list(map(int,df2.loc[i]['pointRec'].split(':')))
        pl.append(p)
    logging.info("Players re-constructed")
    
else:
    logging.info("Creating Table")
    engine.execute("""
    CREATE TABLE PLAYER_DATA (
    id INT AUTO_INCREMENT PRIMARY KEY, 
    name VARCHAR(255), 
    skill INT(10), 
    variance INT(10),
    pointRec VARCHAR(255),
    totalPoints INT(10),
    pointLimit INT(10))""")
    
    logging.info("Table created. Creating Players.")
    pl,tab = synthSportPlayer.generatePlayers(32)
    tab2=tab.copy()
    tab2.sort_index(inplace=True)
    tab2.drop('week_-1',axis=1,inplace=True)
    tab2["pointRec"]=''
    tab2["totalPoints"]=0
    tab2["pointLimit"]=10
    tab2.to_sql("PLAYER_DATA",engine,index=False,if_exists='append')
    logging.info("Players created")
    

In [47]:
df = pd.read_sql('SHOW TABLES', con=engine)
if 'RANKING_DATA' in df['Tables_in_SSPDB'].values:
    logging.info('Table already exists')
    df3 = pd.read_sql('SELECT * FROM RANKING_DATA', con=engine)
    
else:
    logging.info("Creating Table")
    engine.execute("""
    CREATE TABLE RANKING_DATA (
    rank_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    points INT(10),
    week VARCHAR(255))
    """)
    logging.info("Table created.")
    

In [48]:
pd.read_sql('SHOW TABLES', con=engine)

,Tables_in_SSPDB
0,PLAYER_DATA
1,RANKING_DATA


In [6]:
ls=synthSportPlayer.liveSeason(numPlayers=len(pl),players=pl)

In [7]:
ls.playerSum

,name,skill,variance,week_-1
4,4,94,10,0
12,12,89,10,0
0,0,87,10,0
10,10,81,10,0
8,8,71,10,0
14,14,68,10,0
5,5,67,10,0
11,11,56,10,0
9,9,54,10,0
6,6,52,10,0


In [22]:
ls.playSeason()

False

In [23]:
ls.currentTourn.matchRec

[[1, 1, '0', 0, 83, '3', 0, 13, '0'],
 [1, 2, '2', 0, 17, '1', 0, 46, '1'],
 [1, 3, '14', 0, 59, '9', 0, 32, '14'],
 [1, 4, '4', 0, 110, '10', 0, 70, '4'],
 [1, 5, '7', 0, 0, '13', 0, 15, '13'],
 [1, 6, '6', 0, 48, '8', 0, 76, '8'],
 [1, 7, '11', 0, 35, '15', 0, 7, '11'],
 [1, 8, '12', 0, 95, '5', 0, 56, '12'],
 [2, 1, '0', 0, 95, '1', 0, 41, '0'],
 [2, 2, '14', 0, 53, '4', 0, 88, '4'],
 [2, 3, '13', 0, 0, '8', 0, 81, '8'],
 [2, 4, '11', 0, 46, '12', 0, 107, '12'],
 [3, 1, '0', 0, 75, '4', 0, 110, '4'],
 [3, 2, '8', 0, 67, '12', 0, 87, '12'],
 [4, 1, '4', 0, 97, '12', 0, 80, '4']]

In [26]:
ls.currentTournComplete

True

In [38]:
tab=ls.playerSum.iloc[:,[0,-1]].copy()
tab['week']=tab.columns[-1]
tab.rename(columns={tab.columns[-2]: "points"},inplace=True)
tab.to_sql("RANKING_DATA",engine,index=False,if_exists='append')

In [50]:
df3 = pd.read_sql('SELECT * FROM RANKING_DATA', con=engine)
df3

,rank_id,name,points,week
0,1,4,25,week_0
1,2,12,20,week_0
2,3,0,15,week_0
3,4,10,5,week_0
4,5,8,15,week_0
5,6,14,10,week_0
6,7,5,5,week_0
7,8,11,10,week_0
8,9,9,5,week_0
9,10,6,5,week_0


In [ ]:
columns=["Round", "Match",
        "player1_id","player1_rnkPoints","player1_perform",
        "player2_id","player2_rnkPoints","player2_perform","winner_id"]

## reference codes

In [ ]:
# js = ':'.join(map(str,[10,12]))  # joins list of int into one str
# list(map(int,js.split(':')))     # splits list of join int str to list of int

In [ ]:
# engine.execute("DROP TABLE PLAYER_DATA") # TO DROP TABLE

In [21]:
# os.environ['API_USER'] = 'username'
# os.environ['API_PASSWORD'] = 'secret'

In [ ]:
## To hit the api
# import requests

# # HOST = os.getenv('HOST')                #insert from pod def, should be 'sspengine'
# HOST='localhost'
# url = 'http://{host}:2001/progress'.format(host=HOST)
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# r = requests.put(url, headers=headers)
